In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
import plotly.express as px
plt.style.use('fivethirtyeight')
import boto3

In [ ]:
# ***Aquí se puede poner el archivo como variable
df_raw = pd.read_csv('./raw_data/Bakery_sales.csv', index_col=0)
df_raw.head()

,date,time,ticket_number,article,Quantity,unit_price
0,2021-01-02,08:38,150040.0,BAGUETTE,1.0,"0,90 €"
1,2021-01-02,08:38,150040.0,PAIN AU CHOCOLAT,3.0,"1,20 €"
4,2021-01-02,09:14,150041.0,PAIN AU CHOCOLAT,2.0,"1,20 €"
5,2021-01-02,09:14,150041.0,PAIN,1.0,"1,15 €"
8,2021-01-02,09:25,150042.0,TRADITIONAL BAGUETTE,5.0,"1,20 €"


In [ ]:
df_raw.shape

(234005, 6)

In [ ]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 234005 entries, 0 to 511395
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   date           234005 non-null  object 
 1   time           234005 non-null  object 
 2   ticket_number  234005 non-null  float64
 3   article        234005 non-null  object 
 4   Quantity       234005 non-null  float64
 5   unit_price     234005 non-null  object 
dtypes: float64(2), object(4)
memory usage: 12.5+ MB


In [ ]:
# Nulos por columna
df_raw.isna().sum()

date             0
time             0
ticket_number    0
article          0
Quantity         0
unit_price       0
dtype: int64

In [ ]:
# Duplicados
df_raw.duplicated().sum()

1210

In [ ]:
# Elimina duplicados
df_raw = df_raw.drop_duplicates()
df_raw.shape

(232795, 6)

In [ ]:
# Renombra columnas
df_raw = df_raw.rename(columns = {'date' : 'Date',
                         'time' : 'Time',
                          'ticket_number' : 'Transaction_id',
                          'article' : 'Menu',
                          'unit_price' : 'Price'})

# Convierte 'Menu' a mayúsculas y minúsculas
df_raw['Menu'] = df_raw['Menu'].str.title()

# Arregla formato de valores 'Quantity' y 'Price'
df_raw = df_raw.replace(to_replace = {'Quantity' : {'.' : ' '}, # np.NaN
                              'Price' : {'€' : ' ' , ',' : '.' }}, regex=True)

# Convierte tipos de dato de 'Quantity', 'Price', 'Menu' y 'Transaction_id'
df_raw = df_raw.astype({'Quantity' : 'int', 
                'Price' : 'float', 
                'Transaction_id' : 'int',
                'Menu' : 'str'})

df_raw.head()

,Date,Time,Transaction_id,Menu,Quantity,Price
0,2021-01-02,08:38,150040,Baguette,1,0.90
1,2021-01-02,08:38,150040,Pain Au Chocolat,3,1.20
4,2021-01-02,09:14,150041,Pain Au Chocolat,2,1.20
5,2021-01-02,09:14,150041,Pain,1,1.15
8,2021-01-02,09:25,150042,Traditional Baguette,5,1.20


In [ ]:
# Crea columna 'Revenue'
df_raw['Revenue'] = df_raw['Quantity'] * df_raw['Price']

In [ ]:
# Cambia Date a datetime
df_raw['Date'] = pd.to_datetime(df_raw['Date'])

# Separa 'Date' en 'Week_day', 'Day', Month' y 'Year'
df_raw['Week_day'] = df_raw['Date'].dt.weekday
df_raw['Day'] = df_raw['Date'].dt.day
df_raw['Year'] = df_raw['Date'].dt.year
df_raw['Month'] = df_raw['Date'].dt.to_period('M')
df_raw['Quarter'] = df_raw['Date'].dt.to_period('Q')

# 'Week_day' en nombres de días y 'Month' a nombres de meses
df_raw = df_raw.replace(to_replace = {'Week_day' : {0 : 'Monday', 1 : 'Tuesday', 2 : 'Wednesday',
                                            3 : 'Thursday', 4 : 'Friday', 5 : 'Saturday',
                                            6 : 'Sunday'}}, regex=True)

df_raw.head(1)

,Date,Time,Transaction_id,Menu,Quantity,Price,Revenue,Week_day,Day,Year,Month,Quarter
0,2021-01-02,08:38,150040,Baguette,1,0.9,0.9,Saturday,2,2021,2021-01,2021Q1


In [ ]:
df_raw.dtypes 

Date              datetime64[ns]
Time                      object
Transaction_id             int64
Menu                      object
Quantity                   int64
Price                    float64
Revenue                  float64
Week_day                  object
Day                        int32
Year                       int32
Month                  period[M]
Quarter            period[Q-DEC]
dtype: object

In [ ]:
# Separa 'Time' y la convierte en datetime
df_raw['Day_time'] = pd.to_datetime(df_raw['Time']).dt.hour
df_raw['Day_time'].unique()

/var/folders/mn/vp70xqd126nc7g0318l132mh0000gp/T/ipykernel_73711/3584091815.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_raw['Day_time'] = pd.to_datetime(df_raw['Time']).dt.hour


array([ 8,  9, 10, 11, 12, 13, 14, 17, 18, 19,  7, 16, 15, 20],
      dtype=int32)

In [ ]:
# Crea función from hour to day time
def day_time (Day_time):
    if Day_time < 12:
        return 'Morning'
    elif Day_time < 17:
        return 'Afternoon'
    else:
        return 'Evening'
    
# Apply function to the new column    
df_raw['Day_time'] = df_raw.Day_time.apply(day_time)
df_raw.head(1)

,Date,Time,Transaction_id,Menu,Quantity,Price,Revenue,Week_day,Day,Year,Month,Quarter,Day_time
0,2021-01-02,08:38,150040,Baguette,1,0.9,0.9,Saturday,2,2021,2021-01,2021Q1,Morning


In [ ]:
# Ordena columnas para visualización
df_raw.sort_values(by=['Menu', 'Quantity', 'Price', 'Revenue'])

In [ ]:
# Elimina valores '.' y 0 en 'Price'
df_raw = df_raw[~df_raw['Menu'].isin(['.'])].reset_index(drop = True)
df_raw = df_raw[~df_raw['Price'].isin([0])].reset_index(drop = True)
df_raw.shape

(232763, 13)

In [ ]:
df_raw.head()

,Date,Time,Transaction_id,Menu,Quantity,Price,Revenue,Week_day,Day,Year,Month,Quarter,Day_time
0,2021-01-02,08:38,150040,Baguette,1,0.90,0.90,Saturday,2,2021,2021-01,2021Q1,Morning
1,2021-01-02,08:38,150040,Pain Au Chocolat,3,1.20,3.60,Saturday,2,2021,2021-01,2021Q1,Morning
2,2021-01-02,09:14,150041,Pain Au Chocolat,2,1.20,2.40,Saturday,2,2021,2021-01,2021Q1,Morning
3,2021-01-02,09:14,150041,Pain,1,1.15,1.15,Saturday,2,2021,2021-01,2021Q1,Morning
4,2021-01-02,09:25,150042,Traditional Baguette,5,1.20,6.00,Saturday,2,2021,2021-01,2021Q1,Morning


In [ ]:
# Elimina valores negativos
df_raw = df_raw.loc[~((df_raw['Quantity'] <= 0) | (df_raw['Revenue'] <= 0))].reset_index(drop = True)

# Elimina outliers basado en EDA
df_raw = df_raw.loc[~(df_raw['Revenue'] >= 100)].reset_index(drop = True)
df_raw = df_raw.loc[~(df_raw['Quantity'] >= 100)].reset_index(drop = True)
df_raw.shape

(231478, 13)

In [ ]:
#Guarda archivo limpio
df_raw.to_csv("./clean_data/clean_bakery_sales.csv")

In [ ]:
# Manda los datos limpios a S3 en parquet

# Credenciales de AWS
aws_access_key_id = VAR_KEY_ID
aws_secret_access_key = VAR_ACCESS_KEY

# Set the S3 bucket name and file name
bucket_name = 'bakery-project'
csv_file_name = './clean_data/clean_bakery_sales.csv'
parquet_file_name = './clean_data/clean_bakery_sales.parquet'

# Read the CSV file
df = pd.read_csv(csv_file_name)

# Convert the dataframe to Parquet format
df.to_parquet(parquet_file_name)

# Create an S3 client
s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

# Upload the Parquet file to S3
s3.upload_file(parquet_file_name, bucket_name, 'clean_bakery_sales.parquet')